In [1]:
%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple;
import numpy as np;

#import deepLIFT stuff
scriptsDir = os.environ.get("DEEPLIFT_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable DEEPLIFT_DIR to point to the deeplift code (WITHIN the deeplift repo)");
sys.path.insert(0,scriptsDir);
#Make sure the directory is set to import the lab's version of keras
scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)

import keras_conversion as kc
import theano
import models
from blobs import MxtsMode
import deeplift_util

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, CuDNN not available)
/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
reload(kc)
model_weights  = "mar31_diff_and_present_peaks_datasetLeuk_75M_logpval5_diffmethoddeseq_top400000_peaksize1000_jitter50_noGCmatch_softmaxDiffTasks_CustomModel_GraphSoftmax_currentBest_epoch_5_modelWeights.h5"
model_yaml = "mar31_diff_and_present_peaks_datasetLeuk_75M_logpval5_diffmethoddeseq_top400000_peaksize1000_jitter50_noGCmatch_softmaxDiffTasks_CustomModel_GraphSoftmax_currentBest_epoch_5_modelYaml.yaml"
keras_model = kc.load_keras_model(model_weights, model_yaml,
                                  normalise_conv_for_one_hot_encoded_input=True,
                                  name_of_conv_layer_to_normalise="conv1")

Using Theano backend.


In [3]:
keras_model.inputs['input'].get_config()

{'cache_enabled': True,
 'custom_name': 'input',
 'input_shape': (1, 4, 1000),
 'name': 'Layer',
 'trainable': True}

In [4]:
keras_model.nodes['softmax_for_col_LMPPvLSC_dense'].get_config()

{'W_constraint': None,
 'W_learning_rate_multiplier': None,
 'W_regularizer': None,
 'activation': 'linear',
 'activity_regularizer': None,
 'b_constraint': None,
 'b_learning_rate_multiplier': None,
 'b_regularizer': None,
 'cache_enabled': True,
 'custom_name': 'softmax_for_col_LMPPvLSC_dense',
 'init': 'glorot_uniform',
 'input_dim': None,
 'name': 'Dense',
 'output_dim': 3,
 'trainable': True}

In [5]:
keras_model.nodes.keys()

['conv1',
 'activ1',
 'pool1',
 'conv2',
 'activ2',
 'pool2',
 'flatten1',
 'fc1',
 'activ3',
 'drop1',
 'fc2',
 'activ4',
 'drop2',
 'binary_peaks_dense',
 'binary_peaks_activ',
 'softmax_for_col_LMPPvLSC_dense',
 'softmax_for_col_LMPPvLSC_activ',
 'softmax_for_col_HSCvGMP_dense',
 'softmax_for_col_HSCvGMP_activ',
 'softmax_for_col_LMPPvLeuk_dense',
 'softmax_for_col_LMPPvLeuk_activ',
 'softmax_for_col_MPPvMonocyte_dense',
 'softmax_for_col_MPPvMonocyte_activ',
 'softmax_for_col_LSCvLeuk_dense',
 'softmax_for_col_LSCvLeuk_activ',
 'softmax_for_col_CLPvNKcell_dense',
 'softmax_for_col_CLPvNKcell_activ',
 'softmax_for_col_MPPvBcell_dense',
 'softmax_for_col_MPPvBcell_activ',
 'softmax_for_col_CLPvCD4_dense',
 'softmax_for_col_CLPvCD4_activ',
 'softmax_for_col_GMPvLSC_dense',
 'softmax_for_col_GMPvLSC_activ',
 'softmax_for_col_MPPvMEP_dense',
 'softmax_for_col_MPPvMEP_activ',
 'softmax_for_col_MPPvGMP_dense',
 'softmax_for_col_MPPvGMP_activ',
 'softmax_for_col_GMPvMonocyte_dense',
 'soft

In [37]:
import keras_conversion as kc
import models
reload(models)
reload(kc)
import blobs
reload(blobs)
from blobs import MxtsMode
deeplift_model = kc.convert_graph_model(keras_model, mxts_mode=MxtsMode.Gradient)

In [8]:
#let's forward prop a bunch of random sequences through it
random_sequences = np.random.uniform(size=(10,1,4,1000))

In [ ]:
#will randomly check a couple of the outputs
outputs_to_check = ['softmax_for_col_HSCvBcell_activ', 'softmax_for_col_HSCvErythroid_activ','binary_peaks_activ']

In [ ]:
import theano
#compile theano functions to compute outputs for keras model
keras_theano_funcs = [
    theano.function([keras_model.inputs['input'].input],
                    keras_model.nodes[output_layer_name].get_output(),
                    allow_input_downcast=True)
    for output_layer_name in outputs_to_check
]

In [ ]:
#compile theano funcs to compute otuputs for deeplift model
deeplift_theano_funcs = [
    theano.function([deeplift_model.get_name_to_blob()['input'].get_activation_vars()],
                    deeplift_model.get_name_to_blob()[output_layer_name].get_activation_vars(),
                    allow_input_downcast=True) for output_layer_name in outputs_to_check
]

In [ ]:
#get the fprop results for both keras and deeplift
import deeplift_util
#run the functions on all the inputs
keras_results, deeplift_results = [[deeplift_util.run_function_in_batches(
                                     func, input_data_list=[random_sequences],
                                     batch_size=10, progress_update=None) for func in funcs]
                                    for funcs in [keras_theano_funcs, deeplift_theano_funcs]]

In [ ]:
#verify that they differ by tiny amounts
for keras_result, deeplift_result in zip(keras_results, deeplift_results):
    print("for single seq",np.sum(np.abs(np.array(keras_result[0])-np.array(deeplift_result[0]))))
    print("across all seq",np.sum(np.abs(np.array(keras_result)-np.array(deeplift_result))))

In [41]:
#make sure multiplier computation and stuff works
output_layer_name = 'softmax_for_col_HSCvBcell_dense'
input_layer_name = 'input'

In [42]:
direct_gradient_function = theano.function(
                            [deeplift_model.get_name_to_blob()['input'].get_activation_vars()],
                            theano.tensor.grad(
                             theano.tensor.sum(deeplift_model.get_name_to_blob()[output_layer_name].get_activation_vars()[:,2]),
                                               deeplift_model.get_name_to_blob()[input_layer_name].get_activation_vars()),
                            allow_input_downcast=True)

In [43]:
multipliers_func = deeplift_model.get_target_multipliers_func(
                 find_scores_layer_name=input_layer_name,
                 pre_activation_target_layer_name=output_layer_name)

In [44]:
multipliers = multipliers_func(task_idx=2, input_data_list=[random_sequences], batch_size=10, progress_update=200)

Done 0


In [45]:
gradients_direct = deeplift_util.run_function_in_batches(direct_gradient_function,
                                                         input_data_list=[random_sequences], batch_size=1, progress_update=None)

In [46]:
np.sum(np.abs(np.array(multipliers)-np.array(gradients_direct)))

0.00014297609